![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_DistilBertForZeroShotClassification.ipynb)

## Import ONNX DistilBertForZeroShotClassification  models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `DistilBertForZeroShotClassification ` is only available since in `Spark NLP 5.2.4` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import BERT models trained/fine-tuned for zero shot classification via `DistilBertForZeroShotClassification` or `TFDistilBertForZeroShotClassification`. These models are usually under `Zero-Shot Classification` category and have `distilbert` in their labels
- Reference: [TFDistilBertForZeroShotClassification](https://huggingface.co/docs/transformers/en/model_doc/distilbert#transformers.TFDistilBertForSequenceClassification)
- Some [example models](https://huggingface.co/models?pipeline_tag=zero-shot-classification&sort=downloads&search=distilbert)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- Albert uses SentencePiece, so we will have to install that as well

In [1]:
!pip install -q --upgrade transformers[onnx]==4.48.3 optimum onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [typeform/distilbert-base-uncased-mnli](https://huggingface.co/typeform/distilbert-base-uncased-mnli) model from HuggingFace as an example and load it as a `ORTModelForSequenceClassification`, representing an ONNX model.
- In addition to `TFDistilBertForTokenClassification` we also need to save the `DistilBertTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [ ]:
from transformers import DistilBertTokenizer
from optimum.onnxruntime import ORTModelForSequenceClassification

MODEL_NAME = 'typeform/distilbert-base-uncased-mnli'
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForSequenceClassification.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

Let's have a look inside these two directories and see what we are dealing with:

In [4]:
!ls -l {EXPORT_PATH}

total 261928
-rw-r--r-- 1 root root       753 Jun 14 00:55 config.json
-rw-r--r-- 1 root root 267961213 Jun 14 00:55 model.onnx
-rw-r--r-- 1 root root       125 Jun 14 00:55 special_tokens_map.json
-rw-r--r-- 1 root root      1278 Jun 14 00:55 tokenizer_config.json
-rw-r--r-- 1 root root    231508 Jun 14 00:55 vocab.txt


- As you can see, we need to move `vocabs.txt` from the tokenizer to assets folder which Spark NLP will look for
- We also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [5]:
!mkdir {EXPORT_PATH}/assets

labels = ort_model.config.label2id
labels = sorted(labels, key=labels.get)

with open(EXPORT_PATH + '/assets/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

!mv {EXPORT_PATH}/vocab.txt {EXPORT_PATH}/assets

In [6]:
!ls -lR {EXPORT_PATH}

onnx_models/typeform/distilbert-base-uncased-mnli:
total 261704
drwxr-xr-x 2 root root      4096 Jun 14 00:55 assets
-rw-r--r-- 1 root root       753 Jun 14 00:55 config.json
-rw-r--r-- 1 root root 267961213 Jun 14 00:55 model.onnx
-rw-r--r-- 1 root root       125 Jun 14 00:55 special_tokens_map.json
-rw-r--r-- 1 root root      1278 Jun 14 00:55 tokenizer_config.json

onnx_models/typeform/distilbert-base-uncased-mnli/assets:
total 232
-rw-r--r-- 1 root root     32 Jun 14 00:55 labels.txt
-rw-r--r-- 1 root root 231508 Jun 14 00:55 vocab.txt


In [7]:
!cat {EXPORT_PATH}/assets/labels.txt

ENTAILMENT
NEUTRAL
CONTRADICTION

Voila! We have our `vocab.txt` and `labels.txt` inside assets directory

## Import and Save DistilBertForZeroShotClassification in Spark NLP


- **Install and set up Spark NLP in Google Colab**
  - This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [8]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 47.9 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [9]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

Spark NLP version: 5.5.3
Apache Spark version: 3.5.4


- Let's use `loadSavedModel` functon in `DistilBertForZeroShotClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `DistilBertForZeroShotClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.


In [10]:
from sparknlp.annotator import DistilBertForZeroShotClassification

zero_shot_classifier = DistilBertForZeroShotClassification.loadSavedModel(
    EXPORT_PATH,
    spark
    )\
    .setInputCols(["document", "token"]) \
    .setOutputCol("class") \
    .setCandidateLabels(["urgent", "mobile", "travel", "movie", "music", "sport", "weather", "technology"])

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [11]:
zero_shot_classifier.write().overwrite().save("./{}_spark_nlp_onnx".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [12]:
!rm -rf {EXPORT_PATH}

Awesome 😎  !

This is your DistilBertForZeroShotClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [13]:
! ls -l {MODEL_NAME}_spark_nlp_onnx

total 261732
-rw-r--r-- 1 root root 268002237 Jun 14 00:57 distilbert_classification_onnx
drwxr-xr-x 4 root root      4096 Jun 14 00:57 fields
drwxr-xr-x 2 root root      4096 Jun 14 00:57 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny DistilBertForZeroShotClassification model 😊

In [14]:
zero_shot_classifier_loaded = DistilBertForZeroShotClassification.load("./{}_spark_nlp_onnx".format(MODEL_NAME))\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")

You can see what labels were used to train this model via `getClasses` function:

In [15]:
zero_shot_classifier_loaded.getClasses()

['NEUTRAL', 'CONTRADICTION', 'ENTAILMENT']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [16]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols("document") \
    .setOutputCol("token")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    zero_shot_classifier_loaded
])

texts = [
    ["I have a problem with my iphone that needs to be resolved asap!!"],
    ["Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app."],
    ["I have a phone and I love it!"],
    ["I really want to visit Germany and I am planning to go there next year."],
    ["Let's watch some movies tonight! I am in the mood for a horror movie."],
    ["Have you watched the match yesterday? It was a great game!"],
    ["We need to harry up and get to the airport. We are going to miss our flight!"]
]

df = spark.createDataFrame(texts, ["text"])
results = pipeline.fit(df).transform(df)

results.select("text", "class.result").show(truncate=False)

+--------------------------------------------------------------------------------------------------------------+------------+
|text                                                                                                          |result      |
+--------------------------------------------------------------------------------------------------------------+------------+
|I have a problem with my iphone that needs to be resolved asap!!                                              |[mobile]    |
|Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app.|[technology]|
|I have a phone and I love it!                                                                                 |[mobile]    |
|I really want to visit Germany and I am planning to go there next year.                                       |[travel]    |
|Let's watch some movies tonight! I am in the mood for a horror movie.                                         |[weath

That's it! You can now go wild and use hundreds of `DistilBertForZeroShotClassification` models from HuggingFace 🤗 in Spark NLP 🚀
